<a href="https://colab.research.google.com/github/wiz124/chem169-git/blob/main/Li_Harry_RID_015_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
#Exercise 0
import h5py
import numpy as np
import pandas as pd

prot_embeddings={}
with h5py.File('per-protein.h5','r') as f:
  print(len(f.keys()))
  prot_id=list(f.keys())
  table=[]
  for key in f.keys():
    prot_embeddings[key]=f[key][:]
    table.append({
        'ref_id':key,
        'embedding':prot_embeddings[key]
    })

print(prot_embeddings['A0A385XJ53'].shape)
df=pd.DataFrame(table)
for embedding in df['embedding']:
  if embedding.shape!=(1024,):
    print('error')

display(df)

4403
(1024,)


,ref_id,embedding
0,A0A385XJ53,"[0.06116, 0.08203, 0.001612, 0.0397, -0.0204, ..."
1,A0A385XJE6,"[0.04794, 0.12286, 0.007145, 0.01444, 0.02454,..."
2,A0A385XJK5,"[0.0503, 0.0938, -0.04318, -0.04367, -0.029, -..."
3,A0A385XJL2,"[0.03955, 0.0903, -0.01991, 0.009315, -0.04788..."
4,A0A385XJL4,"[0.008354, 0.1595, 0.02472, 0.0475, 0.03152, 0..."
...,...,...
4398,Q93K97,"[0.04385, 0.006367, -0.03177, 0.03647, 0.02083..."
4399,Q9XB42,"[-0.001819, 0.0888, -0.03842, 0.02385, -0.0679..."
4400,Q9Z3A0,"[0.00667, 0.1096, -0.06158, 0.02464, -0.006783..."
4401,U3PVA8,"[0.037, -0.01764, -0.02815, -0.01985, 0.02533,..."


In [22]:
#Exercise 1
from scipy.spatial.distance import cosine

def compute_distance(vec_a, vec_b, metric):
  if metric=='cosine':
    distance=cosine(vec_a, vec_b)
    return float(distance)

  elif metric=='euclidean':
    distance=np.linalg.norm(vec_a-vec_b)
    return float(distance)

  else:
    return -1.0

vec_a=np.array(df['embedding:'][0])
vec_b=np.array(df['embedding:'][1])

print(compute_distance(vec_a, vec_b, 'cosine'))
print(compute_distance(vec_a, vec_b, 'euclidean'))

0.3154296875
1.169921875


Exercise 1

In cosine similarity, 1.0 means the proteins is high similarity and 0.0 means no similarity. Cosine similarity is computed by cosine distance=1-cosine similarity.

In [ ]:
#Exercise 2
query_embedding=df['embedding'][0]
query_id=df['ref_id'][0]

metric='cosine'
query_vec=np.array(query_embedding)
for i,embedding in enumerate(df['embedding']):
  embedding_vec=np.array(embedding)

  distance=compute_distance(query_vec, embedding_vec, metric)
